In [119]:
import os
import numpy as np
import pandas as pd

import shapely as sp
import shapely.geometry as sgeom
import cartopy.io.shapereader as shpreader

from math import *


In [130]:
import folium
import branca.colormap as cm
import json

In [121]:
fname = 'data/townships/myanmar_township_boundaries.shp'

reader = shpreader.Reader(fname)
tsps_geoms = list(reader.geometries())
tsps_records = list(reader.records())


In [122]:
tsps_df = pd.DataFrame([x.attributes for x in tsps_records])

In [123]:
tsps_df.head()

,AREA,DT,DT_PCODE,LABEL2,OBJECTID,SELF_ADMIN,ST,ST_2,ST_PCODE,ST_RG,TS,TS_PCODE,T_NAME_M3,T_NAME_WIN
0,5761.296368,Mohnyin,MMR001D002,Hpakant\n169795,250,,Kachin,Kachin State,MMR001,State,Hpakant,MMR001009,ဖားကန့်,zm;uefU
1,2703.611427,Mongmit,MMR015D008,Mongmit\n61072,163,,Shan (North),Shan State (North),MMR015,State,Mongmit,MMR015017,မိုးမိတ်,rdk;rdwf
2,952.439789,Bago,MMR007D001,Waw\n199032,96,,Bago (East),Bago Region (East),MMR007,Region,Waw,MMR007004,ဝေါ,a0g
3,1918.673414,Pyay,MMR008D001,Paukkhaung\n117164,147,,Bago (West),Bago Region (West),MMR008,Region,Paukkhaung,MMR008002,ပေါက်ခေါင်း,aygufacgif;
4,1178.507634,Pyinoolwin,MMR010D002,Mogoke\n191775,263,,Mandalay,Mandalay Region,MMR010,Region,Mogoke,MMR010011,မိုးကုတ်,rdk;ukwf


In [124]:
def get_index_of_location(lat,lon,geoms):
    for i, t in enumerate(geoms):
        if t.contains(sgeom.Point(lon, lat)):
            return i

In [125]:
# testing a point in Hpakant, should be index 0 in dataframe

get_index_of_tsp_location(25.6787,96.3858,tsp_geoms)

0

In [126]:
def get_attributes_of_tsp_location(lat,lon,geoms,df):
    return df.iloc[get_index_of_location(lat,lon,geoms),:]

In [127]:
# testing a point in Hlaing Tharyar

get_attributes_of_tsp_location(16.8728,96.0685,tsp_geoms,tsps_df)

AREA                       84.2322
DT                  Yangon (North)
DT_PCODE                MMR013D001
LABEL2        Hlaingtharya\n377632
OBJECTID                       179
SELF_ADMIN                        
ST                          Yangon
ST_2                 Yangon Region
ST_PCODE                    MMR013
ST_RG                       Region
TS                    Hlaingtharya
TS_PCODE                 MMR013008
T_NAME_M3               လှိုင်သာယာ
T_NAME_WIN              vSdkifom,m
Name: 130, dtype: object

In [128]:
def viz_point(lat,lon,geoms,df):
    index = get_index_of_tsp_location(25.6787,96.3858,geoms)
    location_result = get_attributes_of_tsp_location(16.8728,96.0685,geoms,df)
    map = folium.Map(location=[lat,lon], zoom_start=12)
    
    html="""
    <strong>Township: </strong>{tsp}<br/>
    <strong>State/Region: </strong>{st}<br/>
    <strong>Pcode: </strong>{pcode}
    """
    table = folium.Html(html.format(
            tsp = location_result['TS'],
            st = location_result['ST'],
            pcode = location_result['TS_PCODE']
        ), script=True)
    popup = folium.Popup(table)
    
    folium.Marker([lat, lon], popup=popup).add_to(map)
    return map

In [129]:
viz_point(16.8728,96.0685,tsp_geoms,tsps_df)